In [17]:
import requests
import pandas as pd
import numpy as np
import math
assist_rate = 0.5
tar_win_rate = 0.7
tar_kda = 3

TARGET = 0 # 0 is tofucookies, 1 is tofu udons

bases = [
    "4894277-TofuCookies",
    "4962808-TofuUdon"
]
saved = bases[TARGET].split("-")[1]

stats_url = f"https://api.paladins.guru/v3/profiles/{bases[TARGET]}/champions"
stats = requests.get(stats_url).json()

champ_url = 'https://api.paladins.guru/v3/champions'
champs = requests.get(champ_url).json()

# Creating Champion ID - Name dictionary
champ_dict = {}
for i in champs:
    if champs[i]['class'] == "Flanker":
        class_text = "Flank"
    else:
        class_text = champs[i]['class']
        
    champ_dict[str(champs[i]['id'])] = {
        'Name': champs[i]['name'],
        'Class': class_text
    }

data = []
for i in stats["champions"]["-1"]:
    champ_template = {}

    # Calculate vars for easy access
    champ_sheet = stats["champions"]["-1"][i]["total"]
    w_left = math.ceil((champ_sheet['wins']*(tar_win_rate - 1) + (tar_win_rate * champ_sheet['losses']))/(1 - tar_win_rate))
    k_Left = math.ceil(-1 * (champ_sheet['kills'] + assist_rate*champ_sheet['assists'] - (tar_kda * champ_sheet['deaths'])))
    if w_left <= 0 and k_Left <= 0:
        kw_left = "-"
    elif w_left <= 0:
        kw_left = k_Left
    elif k_Left <= 0:
        kw_left = f"0/{w_left}"
    else:
        kw_left = k_Left/w_left
    kda = (champ_sheet['kills'] + assist_rate*champ_sheet['assists'])/champ_sheet['deaths']
    cpm = champ_sheet['gold']/champ_sheet['playtime']
    games = champ_sheet['wins'] + champ_sheet['losses']
    wr = champ_sheet['wins']/games

    # Calculate time
    t = champ_sheet['playtime']
    if t < 60:
        playtime = f"{t}m"
    elif t == 60:
        playtime = f"{t}h"
    else:
        playtime = f"{t//60}h {t%60}m"

    # Calculate score
    score = (cpm*wr*kda*np.log(games)) 

    # HACKY CODE TO FIX NULLS
    try:
        healing_fix = champ_sheet['healing']
    except:
        healing_fix = 0

    try:
        self_healing_fix = champ_sheet['self_healing']
    except:
        self_healing_fix = 0

    try:
        obj_fix = champ_sheet['objective_time']
    except:
        obj_fix = 0
    # Create dictionary
    champ_template = {
        'ID': champ_sheet['id'],
        'Score': round(score,3),
        'Name': champ_dict[champ_sheet['id']]['Name'],
        'Role': champ_dict[champ_sheet['id']]['Class'],
        'Games': games,
        'WR': round(wr,3),
        'KDA': round(kda,3),
        'CPM': round(cpm,3),
        'DPM': round(champ_sheet['damage']/champ_sheet['playtime'],3),
        'HPM': round(healing_fix/champ_sheet['playtime'],3),
        'eHPM': round((healing_fix + self_healing_fix)/champ_sheet['playtime'],3),
        'SPM': round(champ_sheet['mitigated']/champ_sheet['playtime'],3),
        'OPM': round(obj_fix/champ_sheet['playtime'],3),
        'W Left': w_left,
        'K Left': k_Left,
        'K/W Left': kw_left,
        'Playtime': playtime,
        'K/M': round(champ_sheet['kills']/champ_sheet['playtime'],3),
        'D/M': round(champ_sheet['deaths']/champ_sheet['playtime'],3),
        'A/M': round(champ_sheet['assists']/champ_sheet['playtime'],3),
        'Dt/M': round(champ_sheet['taken']/champ_sheet['playtime'],3),
        'wD/tD': round(champ_sheet['in_hand']/champ_sheet['damage'],3),
        'sD/tD': round(1 - champ_sheet['in_hand']/champ_sheet['damage'],3),
        'Playtime (Min)': champ_sheet['playtime'],
        'Credits': champ_sheet['gold'],
        'Kills': champ_sheet['kills'],
        'Deaths': champ_sheet['deaths'],
        'Assists': champ_sheet['assists'],
        'Total Damage': champ_sheet['damage'],
        'Weapon Damage': champ_sheet['in_hand'],
        'Taken': champ_sheet['taken'],
        'Shielding': champ_sheet['mitigated'],
        'Healing': healing_fix,
        'Healing (Self)': self_healing_fix,
        'Wins': champ_sheet['wins'],
        'Losses': champ_sheet['losses'],
        'Objective Time': obj_fix
    }
    data.append(champ_template)

df = pd.DataFrame(data)
df = df.sort_values(by=["W Left", "K Left"], ascending=False)
df.to_csv(f'paladins_data_{saved}.csv', index=False)

In [4]:
# IF DUPLICATE VALUE, +1 to one of the dupes to prevent error
from itertools import combinations
data = {
    "TofuAelish": 1687,
    "TofuCookies": 1716,
    "TofuUdon": 1566,
    "nickcagefan423": 1181,
    "ubisoftreject": 1198,
    "BubbIeTea": 1340,
    "GustaBoosta": 1616,
    "TofuShake": 1867,
    "latamplayername": 1461,
    "lowrez999": 1199
}
def check_dup(data):
    iter = sorted(list(data.values()))
    for i in range(len(iter)-1):
        if iter[i] == iter[i+1]:
            print(f"DUPE - {iter[i]}")
            return
    print("no dupe")
    return
def get_complement(tuple):
    total_set = list(data.values())
    for i in tuple:
        if i in total_set:
            total_set.pop(total_set.index(i))
    return total_set
def get_complement_team(values):
    total_set = list(data.keys())
    for i in values:
        if i in total_set:
            total_set.remove(i)
    return total_set
def avg(list):
    return sum(list)/len(list)
def get_key(v):
    return [i for i in data if data[i]==v][0]

check_dup(data)
all_combos = list(combinations(data.values(), 5))
answers = []
for i, v in enumerate(all_combos):
    answers.append((i, abs(avg(v) - avg(get_complement(v)))))
sort_ans = sorted(answers, key=lambda x: x[1])
best_team = all_combos[sort_ans[0][0]]
names = [get_key(i) for i in best_team]

print(f"Difference: {round(sort_ans[0][1],2)}")
print(names)
print(get_complement_team(names))

no dupe
Difference: 0.2
['TofuAelish', 'TofuCookies', 'ubisoftreject', 'GustaBoosta', 'lowrez999']
['TofuUdon', 'nickcagefan423', 'BubbIeTea', 'TofuShake', 'latamplayername']


In [11]:
import pyautogui

# NOTE you must have tofucookies prepopulated in search

tofu_pos = (853, 465) #2nd position
tofu_pos = (853, 420) #2nd position
global_cd = .6
arrow_cd = .25
champ_rows = 2
num_champs_on_last_row = 4

def import_loadouts():
    button_pos = [(1073, 937), (635, 935), (950, 608), (1159, 624), tofu_pos, (861, 842), (842, 943)]
    button_tag = ["clear all", "import", "search by name", "search", "click tofucookies", "import", "save and exit"]
    loadout_pos = [(310, 400), (730, 400), (310, 620), (730, 620), (310, 840), (730, 840)]
    for i in range(len(loadout_pos)):
        # print(f"click loadout {i+1}")
        pyautogui.click(loadout_pos[i][0], loadout_pos[i][1]) # click arrow
        pyautogui.PAUSE = global_cd
        for j in range(len(button_pos)):
            if j == 5: # if on the coords corresponding to clicking "import" button 
                if i == 0:
                    # print("no arrow clicks")
                    pass
                else:
                    for k in range(i):
                        # print(f"arrow click {k}")
                        pyautogui.click(1029, 739) # click arrow
                        pyautogui.PAUSE = arrow_cd
            # print(button_tag[j])
            pyautogui.click(button_pos[j][0], button_pos[j][1])
            pyautogui.PAUSE = global_cd
    # print("completed")
    return

def champ_char_pane_navigate(row, col):
    # print(f"click {col+1} champ on {row+1} row")
    pyautogui.click(120+(120*col), 300+(120*row))
    pyautogui.PAUSE = global_cd
    pyautogui.click(1159, 84) # click loadout menu
    pyautogui.PAUSE = global_cd
    import_loadouts()
    pyautogui.PAUSE = global_cd
    pyautogui.click(47, 45) # return to champ screen
    print(f"done with {col+1} champ on {row+1} row")
    return

def champ_menu_navigate(total_row, total_col):
    for row in range(total_row):
        if row == total_row - 1: # on last row
            for col in range(total_col):
                champ_char_pane_navigate(row, col)
        else:
            for col in range(8):
                champ_char_pane_navigate(row, col)
    print("done w all")
    return

In [ ]:
champ_menu_navigate(2,6)

In [12]:
import_loadouts()

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.